In [1]:
# Quality control of the reads
!fastqc -o . amp_res_1.fastq amp_res_2.fastq

Started analysis of amp_res_1.fastq
Approx 5% complete for amp_res_1.fastq
Approx 10% complete for amp_res_1.fastq
Approx 15% complete for amp_res_1.fastq
Approx 20% complete for amp_res_1.fastq
Approx 25% complete for amp_res_1.fastq
Approx 30% complete for amp_res_1.fastq
Approx 35% complete for amp_res_1.fastq
Approx 40% complete for amp_res_1.fastq
Approx 45% complete for amp_res_1.fastq
Approx 50% complete for amp_res_1.fastq
Approx 55% complete for amp_res_1.fastq
Approx 60% complete for amp_res_1.fastq
Approx 65% complete for amp_res_1.fastq
Approx 70% complete for amp_res_1.fastq
Approx 75% complete for amp_res_1.fastq
Approx 80% complete for amp_res_1.fastq
Approx 85% complete for amp_res_1.fastq
Approx 90% complete for amp_res_1.fastq
Approx 95% complete for amp_res_1.fastq
Analysis complete for amp_res_1.fastq
Started analysis of amp_res_2.fastq
Approx 5% complete for amp_res_2.fastq
Approx 10% complete for amp_res_2.fastq
Approx 15% complete for amp_res_2.fastq
Approx 20% c

In [2]:
# Trimming low quality reads
!trimmomatic PE -phred33 amp_res_1.fastq amp_res_2.fastq amp_res_1_trim.fastq amp_res_1_unpaired.fastq amp_res_2_trim.fastq amp_res_2_unpaired.fastq  SLIDINGWINDOW:10:20 LEADING:20 TRAILING:20 MINLEN:20


TrimmomaticPE: Started with arguments:
 -phred33 amp_res_1.fastq amp_res_2.fastq amp_res_1_trim.fastq amp_res_1_unpaired.fastq amp_res_2_trim.fastq SLIDINGWINDOW:10:20 LEADING:20 TRAILING:20 MINLEN:20
Multiple cores found: Using 4 threads
Input Read Pairs: 455876 Both Surviving: 452888 (99,34%) Forward Only Surviving: 2941 (0,65%) Reverse Only Surviving: 41 (0,01%) Dropped: 6 (0,00%)
TrimmomaticPE: Completed successfully


In [ ]:
!bwa index 

In [5]:
# Align reads to reference
!bwa mem reference_genomic.fna amp_res_1_trim.fastq amp_res_2_trim.fastq > alignment.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 106290 sequences (10000002 bp)...
[M::process] read 108208 sequences (10000042 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (9, 51226, 0, 22)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (145, 185, 230)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 400)
[M::mem_pestat] mean and std.dev: (189.29, 63.57)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 485)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation RR...
[M::mem_pestat] (25, 50, 75) percentile: (114, 158, 263)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 561)
[M::mem_pestat] mean and std.dev: (131.82, 63.27)
[M::mem_pestat] low and high boundaries for pro

[M::mem_process_seqs] Processed 107246 reads in 3.201 CPU sec, 3.019 real sec
[M::process] read 38166 sequences (3590317 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (12, 50331, 0, 10)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (68, 135, 213)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 503)
[M::mem_pestat] mean and std.dev: (120.09, 68.06)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 648)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (143, 183, 229)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 401)
[M::mem_pestat] mean and std.dev: (188.26, 63.54)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 487)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation RR

In [7]:
# Compressing .sam file
!samtools view -S -b alignment.sam > alignment.bam

In [9]:
# Basic statistics
!samtools flagstat alignment.bam

891635 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
257 + 0 supplementary
0 + 0 duplicates
890569 + 0 mapped (99.88% : N/A)
891378 + 0 paired in sequencing
445689 + 0 read1
445689 + 0 read2
887530 + 0 properly paired (99.57% : N/A)
889384 + 0 with itself and mate mapped
928 + 0 singletons (0.10% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [15]:
# Sorting .bam file
!samtools sort alignment.bam -o alignment_sorted.bam

In [10]:
# Indexing .bam file
!samtools index alignment_sorted.bam

In [11]:
# Making .mpileup file
!samtools mpileup -f reference_genomic.fna alignment_sorted.bam > my.mpileup

[mpileup] 1 samples in 1 input files


In [13]:
# Variant calling
!varscan mpileup2snp my.mpileup --min-var-freq 0.50 --variants --output-vcf 1 > VarScan_results.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.5
Min avg qual:	15
P-value thresh:	0.01
Reading input from my.mpileup
4641476 bases in pileup file
9 variant positions (6 SNP, 3 indel)
1 were failed by the strand-filter
5 variant positions reported (5 SNP, 0 indel)


In [16]:
# Preparing data for IGV
!cp reference_genomic.fna reference_genomic.fasta
!samtools faidx reference_genomic.fasta